In [1]:
import re, subprocess, pyodbc 
import numpy as np # linear algebra 
from sqlalchemy import func, create_engine
from config import mongoConn, connstr  
import pandas as pd

In [2]:
engine = create_engine(connstr)
conn = engine.connect()
query = """ \
SELECT * FROM vw_MetricsCompositeKeys WHERE PK_FORM = '2021-A-SAOP'
""" 
df = pd.read_sql(query,con=conn)  
df.head(2)

,ID,PK_Form,MAX_PK_ReportingCycle,MIN_Q_PK,MAX_Q_PK,MAX_Q_PK_STAG,QOFFSET,MIN_QG_PK,MAX_QG_PK,MAX_QG_PK_STAG,...,GroupDesc,QPK,IdText,QuestionTypeCode,SortPos,QuestionTypeId,description,FK_PickList,QTEXT,PK
0,1-0-,2021-A-SAOP,106,24600,25098,25108,0,2900,2916,2926,...,Section 1: General Privacy Program Requirements,24600,,INSTRUCTION,0,62,Instruction (Text Only),None,"Note:<BR />If ""No"" is selected for metric 1a, ...",24600
1,1-1-1a,2021-A-SAOP,106,24600,25098,25108,1,2900,2916,2926,...,Section 1: General Privacy Program Requirements,24601,1a,YN,1,1,Yes / No,None,Has the head of the agency designated a Senior...,24601


In [ ]:
def subtype(t):
    t = re.sub('.*Free Text.*', 'TXT', t )  
    t = re.sub('.*DATE.*', 'TXT', t ) 
    t = re.sub('.*Select All.*', 'MULTICHECKBOX', t )  
    t = re.sub('.*PICK.*|.*Select.*', 'PICK', t )  
    t = re.sub('.*UNK.*', 'YNA', t )  
    t = re.sub('.*Y N.*', 'YN', t )   
    return t

In [ ]:
def processdf(df):
    df=df.fillna('~')
    df=df.loc[ df.ID != '~' ]
    df.ID = df.ID.astype(str)
    df.reset_index(drop=True, inplace=True) 
    df["SORDER"] = df.index 
    df["TYPE"] = df.TYPE.apply(lambda x: subtype(x) )
    #df["SUBTYPE"] = df.TYPE.apply(lambda x: subtype(x) )
    df["QTEXT"] = df.QTEXT.apply(lambda x: x.replace('\'','`') ) 
    df["ID"] = df.ID.apply(lambda x: x.replace('\'','') ) 
    return df

In [ ]:
df_mas = pd.DataFrame();
for i in range(2,4):
    df = pd.read_csv(fr'c:\Users\Tim\Documents\1802\s{i}.csv')  
    df['Section'] = i
    df['OFFSET'] = 50 * (i-1)
    df=processdf(df)
    df_mas = pd.concat([df_mas, df], axis=0, ignore_index=True) 
    
df_mas["QPK"] = df_mas.index + 22800 + df_mas.OFFSET
df_mas.drop(columns=['OFFSET'], inplace=True)
df_mas = pd.merge(df_mas , qtypes, left_on='TYPE', right_on='QCODE', how='left',   )
df_mas["DBINSERT"] = "" 
df_mas["ASPX"] = "" 
df_mas.sort_values( by='QPK', inplace=True)
df_mas.reset_index(drop=True, inplace=True)
df_mas[1:2]

In [ ]:
ins = ''
aspx = ''
pickseed = 324
for i,r in df_mas[:].iterrows():
    df_mas.loc[i, 'DBINSERT'] = f"({r.QPK}, @FormName, @PK_QGroup, {r.SORDER}, N'{r.ID}', NULL, NULL, NULL, {r.QTID}, 1, NULL, 0, NULL, NULL, NULL, N'{r.QTEXT}',  NULL),"
    ins = ins + df_mas.loc[i, 'DBINSERT'] + '\n'  
    metric = r.ID.replace('.','_')
    if r.TYPE == 'PICK':
        pickseed = pickseed + 1    
    try:
        s = open(f'C:\\_som\\_src\\{r.QCODE}.aspx', 'r').read()
        s = s.replace('{0}',str(r.QPK)).replace('{1}',str(metric)).replace('{4}',str(metric)+ ' : ' +str(r.QPK)+ '\n\t' +str(r.QTEXT)).replace('{5}','')
        s = s.replace('{2}',str(160))
        df_mas.loc[i, 'ASPX'] = s
        aspx = aspx + df_mas.loc[i, 'ASPX'] + '\n\n' 
    except : 
        pass
 
df_mas.to_csv(r"c:\Users\Tim\Documents\1802\BOD_2021_Metrics.csv", index='false')
aspx = re.sub('_ShowRow','ShowRow', aspx) 
with open(r'c:\Users\Tim\Documents\1802\DB_Update7.34_BOD_2021_READONLY.sql', 'w') as file:
  file.write(ins)
file.close()
with open(r'c:\Users\Tim\Documents\1802\BOD_2021_ASPX_READONLY.aspx', 'w') as file:
  file.write(aspx)
file.close()

df_mas[:]

In [ ]:
sr=''
for i,r in df_mas.iterrows():
    sr = sr + showrow.replace('{0}', r.ID.replace(".","_"))
print(sr)

In [ ]:
df_picks = pd.DataFrame(); 
df = pd.read_csv(fr'c:\Users\Tim\Documents\1802\s1.csv')   
df = df.fillna('~')
df=df.loc[ df.TYPE != '~' ] 
df=df.loc[ df.METRIC.str.contains('PL-')  ] 
pickseed = 3980
sorder=0
METRIC=df.iloc[0]['METRIC']
for i,r in df.iterrows():

    if METRIC != r.METRIC:
        sorder=0
        METRIC=r.METRIC 
    pickseed = pickseed + 1 
    sorder = sorder + 1 
    TYPE = re.sub('\s\s', '', r.TYPE)  
    TYPE= re.sub('\(Free Text\)', '', TYPE) 
    # TYPE= re.sub(', ', '', TYPE) 
    TYPE= re.sub('· ', '', TYPE) 
    CodeValue=r.TYPE.upper() 
    CodeValue = re.sub('[^A-Z0-9]', '', CodeValue)   
    if len(CodeValue) > 4 :
        CodeValue = re.sub('[AEIOUY]', '', CodeValue)
        CodeValue=CodeValue[:4]+CodeValue[-4:]
        if re.search('.*Other.*', TYPE):
            CodeValue="OTH"  
    
    TYPE = TYPE.replace(", "," ")
    df.loc[i, 'PK_Picklist'] = f"({pickseed},@PKT, N'{CodeValue}','{TYPE}',{sorder}, GETDATE(), 1),"
    print(df.loc[i, 'PK_Picklist']) 
    
df.head(59)

In [ ]:
 df = pd.read_csv(fr'c:\Users\Tim\Documents\1802\BOD_2021_Metrics.csv')  

In [ ]:
query = """\
SELECT PK_PickList, PK_PickListType, DisplayValue, CodeValue, SortPos, Description, UsageField FROM vwPickLists
WHERE PK_PickListType IN (100,101,102,103,104,105,106,113,114,164,165,166)
""" 
 
dfpicks = pd.read_sql(query,con=conn) 
UsageField=dfpicks.iloc[0]['UsageField']
s=f"\n"
s = s + f"Enum PICK_{UsageField} \t'{dfpicks.iloc[0]['Description']}-{dfpicks.iloc[0]['PK_PickListType']}\n"
for i,r in dfpicks.iterrows():
    if UsageField != dfpicks.iloc[i]['UsageField']:
        UsageField=dfpicks.iloc[i]['UsageField']
        s = s + f"End Enum\n"
        s = s + f"Enum PICK_{UsageField} \t'{dfpicks.iloc[i]['Description']}-{dfpicks.iloc[i]['PK_PickListType']}\n"
    s = s + f"\t{r.CodeValue} = {str(r.PK_PickList)} \t'{str(r.DisplayValue)[:25]}\n" 
s=s+"End Enum\n"
print(s)

In [ ]:
showrow = """\
    Protected _ShowRow_{0} As String
    Public Property ShowRow_{0}() As String
        Get
            Return _ShowRow_{0}
        End Get
        Set(ByVal value As String)
            _ShowRow_{0} = value
        End Set
    End Property
    
"""

In [ ]:
import nltk
stop_words = set(stopwords.words('english'))  

In [ ]:
#df = pd.read_csv(fr'c:\Users\Tim\Documents\1802\HVA_20_21_ID_Mapper.csv')  
#df.to_sql('HVA_IDTEXT_Map', engine,  if_exists='replace', index=False )
 